In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import numpy as np
import math
import cmath

In [2]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
def parse_complex(current):
    try:
        current = current.replace(" ", "").replace("A", "")
        complex_current = complex(current)
        return complex_current
        
    except Exception as e:
        return {"error": str(e)}  # Only if it fails

def complex_to_dict(c):
    return {"real": c.real, "imag": c.imag}

def dict_to_complex(d):
        real = float(d["real"])
        imag = float(d["imag"])
        return complex(real, imag)

def complex_to_string(c):
    return f"{c.real}+{c.imag}j" if c.imag >= 0 else f"{c.real}{c.imag}j"

In [4]:
def compute_sequence_components_tool(Ia, Ib, Ic):
    
    a = complex(-.5, 2.094)

    Ia = parse_complex(Ia)
    Ib = parse_complex(Ib)
    Ic = parse_complex(Ic)
    
    I0 = (Ia + Ib + Ic) / 3
    I1 = (Ia + (a*Ib) + ((a*a)*Ic)) / 3
    I2 = (Ia + ((a*a)*Ib) + (a*Ic)) / 3

    return {
        "I0": complex_to_string(I0),
        "I1": complex_to_string(I1),
        "I2": complex_to_string(I2),
    }

In [5]:
def identify_fault_type_tool(I0, I1, I2):

    I0 = parse_complex(I0)
    I1 = parse_complex(I1)
    I2 = parse_complex(I2)

    magI0 = abs(I0)
    magI1 = abs(I1)
    magI2 = abs(I2)
    
    threshold = .01
    fault_type = ""

    if magI0 < threshold and magI2 < threshold:
        fault_type = "Three-phase fault"
    if magI0 < threshold and magI2 > threshold:
        fault_type = "Line-to-line fault"
    if magI0 > threshold and magI2 > threshold:
        fault_type = "Line-to-ground or Double-line-to-ground fault"
        
    return fault_type

In [6]:
tools = [
{
    "type": "function",
    "function": {
        "name": "compute_sequence_components_tool",
        "description": "Calculate sequence components given phase currents. Call whenever you need to calculate sequence components. Should only be called once",
        "parameters": {
            "type": "object",
            "properties": {
                "Ia": {"type": "string"},
                "Ib": {"type": "string"},
                "Ic": {"type": "string"}
            },
            "required": ["Ia", "Ib", "Ic"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "identify_fault_type_tool",
        "description": "Identify fault type given sequence currents. Call whenever you need to classify fault type",
        "parameters": {
            "type": "object",
            "properties": {
                "I0": {"type": "string"},
                "I1": {"type": "string"},
                "I2": {"type": "string"}
            },
            "required": ["I0", "I1", "I2"]
        }
    }
}
]

In [7]:
def handle_tool_calls(message):
    tool_responses = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
    
        if tool_name == "compute_sequence_components_tool":
            Ia = arguments.get('Ia')
            Ib = arguments.get('Ib')
            Ic = arguments.get('Ic')
            
            result = compute_sequence_components_tool(Ia, Ib, Ic)

            I0 = result["I0"]
            I1 = result["I1"]
            I2 = result["I2"]
            
            tool_response = {
                "role": "tool",
                "content": json.dumps({"Ia": Ia, "Ib": Ib, "Ic": Ic, "I0": I0,"I1": I1, "I2": I2}),
                "tool_call_id": tool_call.id
            }
            
        elif tool_name == "identify_fault_type_tool":
            I0 = arguments.get('I0')
            I1 = arguments.get('I1')
            I2 = arguments.get('I2')
    
            fault_type = identify_fault_type_tool(I0, I1, I2)
            tool_response = {
                "role": "tool",
                "content": json.dumps({"I0": I0, "I1": I1, "I2": I2, "fault_type": fault_type}),
                "tool_call_id": tool_call.id
            }
            
        tool_responses.append(tool_response)
        
    return tool_responses 
                                  
                                   

In [8]:
system_message = \
    "You are an expert power engineer. You are given voltage and current data for a \
    three phase system. Analyze the data and explain what fault has likely occured." 

def get_user_prompt(Ia, Ib, Ic):
    prompt = (
        "Given these three phase currents:\n"
        f"Phase A current: {Ia} A\n"
        f"Phase B current: {Ib} A\n"
        f"Phase C current: {Ic} A\n\n"
        "Analyze the phase currents and classify the fault.\n\n"
        "Answer in this order, including the step numbers indicated:\n"
        "1. Compute sequence currents and display\n"
        "2. Using computed sequence currents, identify fault and display. Also show magnitudes of sequence components\n"
        "Show magnitudes before identifying the fault type and then show the inequality which you used to determine the fault type\n"
        "3. Briefly Explain possible solutions to the fault.\n"
        "\n"
        "\n"
        "Another note, do not explicity write out the steps you are taking.\n"
        "Also, when writing any value, display without putting value in '\(\)' format"
    )
    
    return prompt

<>:20: SyntaxWarning: invalid escape sequence '\('
<>:20: SyntaxWarning: invalid escape sequence '\('
/var/folders/k6/9rxvgg1x49q49jdz4pd13m8c0000gn/T/ipykernel_77265/1507181550.py:20: SyntaxWarning: invalid escape sequence '\('
  "Also, when writing any value, display without putting value in '\(\)' format"


In [9]:
def gradio_fn(Ia, Ib, Ic):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": get_user_prompt(Ia, Ib, Ic)}
    ]

    while True:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        message = response.choices[0].message

        if response.choices[0].finish_reason != "tool_calls":
            break

        messages.append(message)

        tool_responses = handle_tool_calls(message)
        messages.extend(tool_responses)
        
    stream = openai.chat.completions.create(
        model = MODEL,
        messages=messages,
        stream = True)

    stream_response = ""
    for chunk in stream:
        stream_response += chunk.choices[0].delta.content or ''
        yield stream_response

In [10]:
interface = gr.Interface(
    fn=gradio_fn,
    inputs=[
        gr.Textbox(label="Current A (A)"),
        gr.Textbox(label="Current B (A)"),
        gr.Textbox(label="Current C (A)"),
    ],
    outputs=gr.Textbox(label="Analysis Output", lines=28),
    title="Power System Fault Diagnosis Assistant",
    description="Enter current readings during a fault to analyze likely cause",
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
